<a href="https://colab.research.google.com/github/AnirudhDesai777/legendary-enigma/blob/master/eeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Intalling Pyeeg
!pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-jbg_2xu3
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-jbg_2xu3
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28122 sha256=df73fd069ac763c478e476810ffa9d2de5b848b6913c8511b7e56e07972948d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nwiaf72/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg


In [3]:
import numpy as np
from sklearn.preprocessing import normalize
import pickle as pickle
import pandas as pd
import pyeeg as pe
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,BatchNormalization,Activation

In [4]:
#Mounting drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
def getPSD(sub):
    # channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
    channel = [2, 3, 4, 6, 7, 11, 15, 17, 20, 24, 25, 27, 28, 29]
    band = [4,8,12,16,25,45] #5 bands
    window_size = 256 #Averaging band power of 2 sec
    step_size = 16 #Each 0.125 sec update once
    sample_rate = 70 #Sampling rate of 128 Hz
    temp = []
    with open(r'/content/drive/My Drive/data_preprocessed_python/s'+sub+'.dat','rb') as file:
        
        subject = pickle.load(file, encoding='latin1')
        
        for i in range(1,40):   # trials
            raw_data = subject['data'][i]
            raw_labels = subject['labels'][i]
            start = 0
            while start + window_size < len (raw_data[i]):
                temp_array = []
                temp_data = []
                for j in channel:
                    X = raw_data[j][start: start + window_size]
                    Y = pe.bin_power(X, band, sample_rate)
                    temp_data = temp_data + list(Y[0])
                temp_array.append(np.array(temp_data))
                temp_array.append(raw_labels)
                temp.append(np.array(temp_array))
                start = start + step_size
        temp = np.array(temp)
        np.save('out\s' + sub, temp, allow_pickle=True, fix_imports=True)

In [6]:
subjectList=['01','02']
# for i in range(0,32):
#     if i < 10:
#         subjectList.append('0' + str(i))
#     else:
#         subjectList.append(str(i))
for sub in subjectList:
    getPSD(sub)

In [7]:
data= []
label = []
for subjects in subjectList:
  

    with open('/content/out\s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
          data.append(sub[i][0])
          label.append(sub[i][1])
np.save('data', np.array(data), allow_pickle=True, fix_imports=True)
np.save('label', np.array(label), allow_pickle=True, fix_imports=True)

In [8]:
df=pd.DataFrame(data=data)
df.to_csv("data.csv",index=False)

df1=pd.DataFrame(data=label)
df1.to_csv("label.csv",index=False)

In [9]:
data=pd.read_csv("data.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69
0,2631.150479,1358.957902,982.274705,955.342880,294.874029,2551.385311,1454.875372,1160.444261,1868.547487,735.610538,2184.991704,1227.991908,637.224650,703.920851,446.074558,2540.103438,1130.444764,905.074784,663.709817,269.865088,2125.004953,1349.397472,1179.832701,1613.622506,124.784937,3017.045150,1133.591773,751.750974,1293.610237,246.359972,2942.386187,917.357900,528.091326,503.508233,244.957645,2500.020852,1010.887319,820.172336,899.212773,324.449719,2370.011435,1355.044789,811.638916,1035.489305,138.105326,1692.855714,1021.895232,562.791069,694.489695,81.809478,2658.606580,1241.722987,950.618845,820.843586,181.958041,2356.922525,1065.201648,559.659018,665.610362,139.443419,3316.131121,1239.898426,866.757418,1079.545210,178.715440,2995.306796,1132.885419,636.610161,851.187212,200.616797
1,2581.405819,1483.656365,868.999122,954.211467,207.157127,2712.013984,1167.513223,1127.247373,1790.977339,165.360472,2137.952410,1076.853752,449.022896,718.317714,109.872722,2324.871247,1144.706774,759.431074,693.737420,434.684496,2382.258738,1278.406931,1106.524307,1669.301133,78.347261,2864.148148,1202.543244,713.670122,1220.464349,171.134552,2684.602716,694.236847,452.038821,492.443596,60.167552,2272.257600,978.493430,849.494194,920.779985,258.220013,2257.514548,1290.011467,833.622924,956.613296,185.404789,1808.641468,1030.016079,580.209842,633.681823,23.591388,2564.086359,1266.424853,822.154165,795.378436,94.973731,2247.243240,944.242893,459.721437,636.732476,140.012426,3103.138653,1167.706063,824.685656,1026.452662,407.743916,2778.124424,1083.610349,562.980483,823.352860,94.360409
2,2558.295428,1610.831205,849.374201,930.752733,89.641656,2685.017537,1404.286630,1089.193772,1809.607435,140.086932,2137.573669,1145.901937,449.204238,719.870940,33.926975,2233.064223,1202.503548,777.563949,775.486863,904.494024,2353.730796,1264.779823,1230.317262,1582.081182,384.827387,2809.183146,1332.937492,725.159856,1198.954433,35.875445,2603.049393,784.931176,449.839612,572.106446,549.609360,2123.128448,994.149127,883.888678,933.947664,567.515704,2144.512682,1262.144109,849.144617,902.341927,489.022355,1717.127153,1050.533318,633.299146,655.749096,363.058518,2378.546453,1248.186272,926.515719,857.430873,557.419808,2209.077061,952.971620,459.738613,566.052414,103.880376,3069.033298,1166.310711,895.660584,1000.482320,387.489479,2523.804396,1059.181888,670.962054,793.328897,293.623898
3,2582.142347,1637.480330,927.589794,908.028093,15.189624,2827.300379,1382.212253,1114.313571,1763.913061,443.838430,2194.569664,1166.239323,434.730027,759.082749,345.898603,2053.273117,1062.547758,738.221718,702.557202,307.856869,2469.579010,1265.779549,1254.009664,1604.353916,69.202789,3397.880592,1429.708838,769.081868,1160.194091,21.013955,2777.223422,760.900793,478.261823,482.406019,104.076370,2255.719655,974.306655,835.678527,850.625649,496.629310,1973.626598,1341.330991,856.488587,950.808709,279.920595,1704.936262,1033.381810,617.749557,613.262895,100.443751,2208.557378,1215.232152,863.495946,829.153716,405.477393,2170.972788,944.812815,440.464652,531.830586,108.144776,3025.074576,1148.269068,875.540992,932.441207,30.044974,2442.494117,1027.526851,640.150561,803.652406,43.832560
4,2416.208172,1661.402028,887.095363,907.451324,200.345635,2859.881193,1329.604465,1097.509021,1689.867430,333.974464,2417.757202,1160.579091,528.398177,662.698281,283.931954,2187.138595,1109.866104,746.658845,661.946026,64.543555,2674.531848,1292.502586,1291.908072,1587.051168,586.294502,3425.566271,1510.852054,754.968074,1194.726720,272.584421,2768.836449,823.946899,473.824722,483.399688,39.856268,2171.955831,954.136812,850.446170,819.243168,260.948272,1852.724462,1329.715186,822.756196,916.225476,91.063499,1674.455280,1074.925170,611.528447,615.839764,153.417477,2202.963790,1275.565874,878

In [10]:
label=pd.read_csv("label.csv")
label

,0,1,2,3
0,8.10,7.31,7.28,8.47
1,8.10,7.31,7.28,8.47
2,8.10,7.31,7.28,8.47
3,8.10,7.31,7.28,8.47
4,8.10,7.31,7.28,8.47
...,...,...,...,...
38059,6.05,6.67,6.94,6.53
38060,6.05,6.67,6.94,6.53
38061,6.05,6.67,6.94,6.53
38062,6.05,6.67,6.94,6.53


In [11]:
y_valence=label.loc[:,'0']
y_valence

0        8.10
1        8.10
2        8.10
3        8.10
4        8.10
         ... 
38059    6.05
38060    6.05
38061    6.05
38062    6.05
38063    6.05
Name: 0, Length: 38064, dtype: float64

In [12]:
x=data
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
print(x)


y_val = to_categorical(y_valence)
y_val

[[ 0.51158284  0.85952642  0.51960918 ... -0.65163891 -0.96421364
  -0.24588982]
 [ 0.45323289  1.25478335 -0.0329439  ... -1.07960455 -1.04095438
  -0.884279  ]
 [ 0.42612464  1.65788966 -0.12867336 ... -0.45197173 -1.12373199
   0.31289756]
 ...
 [ 0.40927513 -1.09130004 -1.119921   ... -0.24660902 -0.71881501
  -0.75968304]
 [ 0.20077041 -1.20279954 -1.08209162 ... -0.33526976 -0.83753312
  -1.1960498 ]
 [ 0.18609228 -1.12560375 -1.14655668 ... -0.60470245 -0.80743894
  -0.74450007]]


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
x = np.reshape(x, (x.shape[0],1,x.shape[1]))

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_val, test_size = 0.2, random_state = 4)

In [15]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,BatchNormalization,Activation
model = Sequential()
model.add(LSTM(512, batch_input_shape = (None, None, x.shape[2]),return_sequences=True))
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(512, activation='relu',return_sequences=True))
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(256,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(LSTM(128,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(64,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(LSTM(32,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Dense(10))
model.add(Activation('sigmoid'))

rmsprop =keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08)
model.compile(loss='mean_squared_error',
                  optimizer=rmsprop,
                  metrics=['accuracy'])
#adam = keras.optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         1193984   
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)        

In [16]:
history = model.fit(x_train, y_train, epochs = 900, batch_size=150,validation_data= (x_test, y_test))

Epoch 1/900
204/204 [==============================] - 5s 27ms/step - loss: 0.2784 - accuracy: 0.1027 - val_loss: 0.2158 - val_accuracy: 0.0775
Epoch 2/900
204/204 [==============================] - 5s 22ms/step - loss: 0.2499 - accuracy: 0.1161 - val_loss: 0.1899 - val_accuracy: 0.0775
Epoch 3/900
204/204 [==============================] - 5s 22ms/step - loss: 0.2252 - accuracy: 0.1265 - val_loss: 0.1925 - val_accuracy: 0.0876
Epoch 4/900
204/204 [==============================] - 5s 22ms/step - loss: 0.2008 - accuracy: 0.1385 - val_loss: 0.1774 - val_accuracy: 0.1363
Epoch 5/900
204/204 [==============================] - 4s 22ms/step - loss: 0.1776 - accuracy: 0.1559 - val_loss: 0.1592 - val_accuracy: 0.2074
Epoch 6/900
204/204 [==============================] - 4s 22ms/step - loss: 0.1565 - accuracy: 0.1553 - val_loss: 0.1407 - val_accuracy: 0.2050
Epoch 7/900
204/204 [==============================] - 5s 22ms/step - loss: 0.1382 - accuracy: 0.1669 - val_loss: 0.1240 - val_accuracy: